In [1]:
from keras.layers import Input, Conv2D, Dense, Flatten
from keras.models import Model
from keras import initializers
import keras

import numpy as np
import imageio

Using TensorFlow backend.


In [2]:
# patch size is larger than the default value of PATCH_SIZE in OpenCV impl. (48),
#   s.t. the blurring can be done with averaging kernel (Conv2D) with valid padding
patch_size = 56  

averaging_kernel_weight_3x3 = 1./9

# as calculated in step1_coordinates_to_dense_layer_weights.py
weights_dense = np.load("./weights/weights_dense_64.npy")

In [3]:
input_shape = (patch_size, patch_size, 1)
input_img = Input(shape=input_shape)

x = Conv2D(1, (3,3), activation="relu", padding="valid", kernel_initializer=initializers.Constant(averaging_kernel_weight_3x3), bias_initializer=keras.initializers.Zeros(), trainable=False)(input_img)
x = Conv2D(1, (3, 3), activation="relu", padding="valid", kernel_initializer=initializers.Constant(averaging_kernel_weight_3x3), bias_initializer=keras.initializers.Zeros(), trainable=False)(x)
x = Conv2D(1, (3, 3), activation="relu", padding="valid", kernel_initializer=initializers.Constant(averaging_kernel_weight_3x3), bias_initializer=keras.initializers.Zeros(), trainable=False)(x)
average_blurred = Conv2D(1, (3, 3), activation="relu", padding="valid", kernel_initializer=initializers.Constant(averaging_kernel_weight_3x3), bias_initializer=keras.initializers.Zeros(), trainable=False)(x)
flattened = Flatten(data_format="channels_last")(average_blurred)
briefed = Dense(weights_dense.shape[1], activation="sigmoid", weights=[weights_dense, np.zeros(weights_dense.shape[1])], trainable=False)(flattened)

brief = Model(input_img, briefed)

brief.compile(optimizer='adadelta', loss="binary_crossentropy")
brief.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 56, 56, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 54, 54, 1)         10        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 1)         10        
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 50, 1)         10        
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 48, 48, 1)         10        
_________________________________________________________________
flatten_1 (Flatten)          (None, 2304)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               1180

In [4]:
# test BRIEF implemented as a NN on a random patch
image = imageio.imread("./images/briefs_gray.bmp")
patch = image[96: 96 + patch_size, 68: 68 + patch_size]
patch_with_chnl_dim = np.expand_dims(patch, axis=-1)
patches = np.expand_dims(patch_with_chnl_dim, axis=0)

descrs = brief.predict(patches)
descr = np.array(np.round(descrs[0]), dtype=np.int32)
print(descr.shape)
print(descr)

(512,)
[1 0 1 1 0 0 0 1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 0 1 1 0 0 1 0 0 1 0 1 0 1 1
 0 1 0 0 1 1 0 0 0 0 0 1 1 0 1 1 1 1 0 0 0 0 1 0 1 1 0 0 0 0 1 0 1 0 0 1 1
 1 1 1 0 0 0 1 0 1 1 0 0 0 1 1 1 0 0 0 0 1 1 1 1 0 1 1 1 1 0 0 1 0 0 1 1 1
 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 0 1 1 0 1 0 1 1 1 0 1 0 1 1 0 1 0
 0 0 0 0 1 0 0 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 0 0 1 1 1 0 1 1 0 1 0 0 0 1 1
 1 1 1 0 1 0 1 0 0 0 1 0 1 0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 1 0 1 1 0 1
 0 1 1 1 0 0 0 1 1 1 1 0 1 0 0 0 1 1 0 1 0 1 1 0 0 1 0 0 0 0 1 1 0 0 0 1 1
 0 0 0 1 0 1 1 1 0 1 0 1 1 0 1 0 1 0 1 1 0 1 1 1 1 0 0 0 1 1 0 1 1 0 0 1 0
 1 1 1 0 0 1 0 1 1 0 0 0 0 1 1 1 0 1 1 1 0 0 1 0 1 0 0 1 1 0 0 0 0 1 1 1 0
 1 1 1 1 1 0 0 0 0 0 1 1 1 1 0 1 0 1 1 0 1 0 1 0 1 0 1 0 0 1 0 1 0 1 0 0 1
 1 0 1 1 1 1 1 0 1 1 1 0 0 1 1 0 1 1 1 1 1 0 1 0 0 0 1 1 0 0 1 0 1 0 0 1 1
 0 0 0 0 0 0 0 1 0 1 1 0 1 1 1 1 0 0 1 1 0 1 1 1 0 1 0 1 1 0 1 1 0 0 0 1 0
 1 1 0 1 1 1 1 0 1 0 0 0 1 1 0 1 0 0 0 1 1 0 1 0 0 1 1 1 0 0 1 0 0 0 0 0 0
 1 0 1 0 0 1 1 0 0